In [12]:
# Imports

import pandas as pd
import qgrid
from sklearn.linear_model import LinearRegression
import os
from multiprocessing import Pool

from tqdm.notebook import tqdm
from highcharts import Highstock

from data_management.database import DataBase

In [13]:
db = DataBase()

In [14]:
# Calculation functions

def performance(series):
    X = series.index.values.reshape(-1,1)
    y = series.values.reshape(-1,1)

    regressor = LinearRegression()  
    regressor.fit(X, y)
    
    perf = regressor.coef_[0][0]
    yearly = ((perf * 250) / series.iloc[-1]) *100
    return yearly

def barcount(series):
    return series.size

In [15]:
# Get contracts
query = """SELECT contract_id, symbol, exchange, name
            FROM contracts"""

conn = db.connect()
cur = conn.cursor()

dfc = pd.read_sql_query(query, conn)

conn.commit()
cur.close()
db.disconnect(conn)

In [20]:
# Pre-filtering of contracts
dfc = dfc.loc[(dfc['name'].str.contains('short', case=False, regex=False)),:]

In [31]:
# Add calculations to instruments

def add_calcs(contract_id):
    # Get quotes
    query = f"""SELECT date, open, high, low, close, volume
                FROM quotes
                WHERE contract_id = {contract_id}"""

    conn = db.connect()
    cur = conn.cursor()

    dfq = pd.read_sql_query(query, conn)

    conn.commit()
    cur.close()
    db.disconnect(conn)

    result = {}
    result['contract_id'] = contract_id
    result['Perf_1w'] = performance(dfq.iloc[-5:]['close'])
    # result['Perf_3m'] = performance(dfq.iloc[-66:]['close'])
    # result['Perf_1y'] = performance(dfq.iloc[-250:]['close'])
    # result['Perf_2y'] = performance(dfq.iloc[-500:]['close'])
    # result['Len'] = len(dfq.iloc[:]['close'])
    return result
    

contract_ids = dfc['contract_id'][:]
with Pool() as p:
    results = p.map(add_calcs, contract_ids)
    
for result in results:
    dfc.loc[dfc['contract_id']==result['contract_id'], 'Perf_1w'] = result['Perf_1w']
    # dfc.loc[dfc['contract_id']==result['contract_id'], 'Perf_3m'] = result['Perf_3m']
    # dfc.loc[dfc['contract_id']==result['contract_id'], 'Perf_1y'] = result['Perf_1y']
    # dfc.loc[dfc['contract_id']==result['contract_id'], 'Perf_2y'] = result['Perf_2y']
    # dfc.loc[dfc['contract_id']==result['contract_id'], 'Len'] = result['Len']

In [32]:
# Post-filtering of contracts

In [33]:
# Build QGrid widget

qgw = qgrid.show_grid(dfc, grid_options={'maxVisibleRows': 30})

In [34]:
# QGrid handlers

def handle_selection_changed(event, widget):
    line_no = event['new'][0]
    build_chart(line_no)
    
qgw.on('selection_changed', handle_selection_changed)


def handle_data_changed(event, widget):
    global dfc
    dfc = qgw.get_changed_df()

qgw.on('filter_changed', handle_data_changed)
qgw.on('sort_changed', handle_data_changed)

In [35]:
# Chart builder

def build_chart(line_no):
    # Get quotes
    contract_id = dfc.iloc[line_no,]['contract_id']
    name = dfc.iloc[line_no,]['name']
    query = f"""SELECT date, open, high, low, close
                FROM quotes
                WHERE contract_id = {contract_id}
                ORDER BY date ASC;"""

    conn = db.connect()
    cur = conn.cursor()
    df = pd.read_sql_query(query, conn)
    conn.commit()
    cur.close()
    db.disconnect(conn)

    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    # df['date'] = (df['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

    chart = Highstock()
    data = df[['date','open','high','low','close']].values.tolist()
    chart.add_data_set(data, series_type='ohlc', name=str(contract_id))
    # todo: add moving average

    options = {
        'chart': {
            'animation': False,
            'height': 800,
            'marginLeft': 50
        },
        'rangeSelector': {
            'selected': 5,
            'buttons': [{
                'type': 'month',
                'count': 1,
                'text': '1m'
            }, {
                'type': 'month',
                'count': 3,
                'text': '3m'
            }, {
                'type': 'month',
                'count': 6,
                'text': '6m'
            }, {
                'type': 'ytd',
                'text': 'YTD'
            }, {
                'type': 'year',
                'count': 1,
                'text': '1y'
            }, {
                'type': 'year',
                'count': 2,
                'text': '2y'
            }, {
                'type': 'all',
                'text': 'All'
            }]
        },
        'navigator': {
            'height': 100
        },
        'title': {
            'text': name
        },
        'plotOptions': {
            'series': {
                'animation': False
            }
        },
        'yAxis': {
            'type': 'logarithmic'
        }
    }
    chart.set_dict_options(options)
    chart.save_file('./screener/website/index')

In [36]:
##### Show QGrid widget

qgw

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…